<a href="https://www.kaggle.com/code/irwanprabowo/healthy-lunch-planner-ai?scriptVersionId=280874396" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# ============================================================
#  Gemini 2.0 Flash + Multi-Agent Healthy Lunch Planner AI
# ============================================================

import json, random, os

# ============================================================
#  GEMINI 2.0 SETUP  (new google.genai SDK)
# ============================================================
USE_GEMINI = False

try:
    from google import genai
    
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "").strip()
    
    if GEMINI_API_KEY:
        client = genai.Client(api_key=GEMINI_API_KEY)
        USE_GEMINI = True

except Exception as e:
    print("Gemini unavailable, running in offline mode:", e)
    USE_GEMINI = False


def call_gemini(prompt: str) -> str:
    """
    Unified Gemini call wrapper with graceful fallback.
    """
    if USE_GEMINI:
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt
            )
            return response.text
        except Exception as e:
            return f"(Gemini error: {e})"

    # Offline fallback mode
    return (
        "Offline Plan: 1) Load meal data. "
        "2) Add ratings. 3) Compute averages. 4) Compare to user goals."
    )


# ============================================================
#  MEMORY COMPONENTS
# ============================================================
class InMemorySessionService:
    def __init__(self): 
        self.sessions = {}

    def get(self, sid): 
        return self.sessions.setdefault(sid, {})

    def set(self, sid, data): 
        self.sessions[sid] = data


class MemoryBank:
    def __init__(self, path="lunch_memory.json"): 
        self.path = path

    def load(self): 
        return json.load(open(self.path)) if os.path.exists(self.path) else {}

    def save(self, data): 
        json.dump(data, open(self.path, "w"), indent=2)


# ============================================================
#  TOOLS
# ============================================================
def csv_tool():
    """Simulated CSV dataset of meals."""
    return [
        {"meal": "Chicken Salad", "calories": 450, "price": 7.5},
        {"meal": "Veggie Wrap", "calories": 380, "price": 6.0},
        {"meal": "Sushi Bento", "calories": 520, "price": 9.0},
        {"meal": "Beef Bowl", "calories": 640, "price": 8.5},
    ]


def openapi_tool(endpoint, params):
    """Mock meal rating API."""
    if endpoint == "/rating":
        return {
            "meal": params.get("meal", "Unknown"),
            "rating": round(random.uniform(3.5, 5.0), 1)
        }
    return {"error": "invalid endpoint"}


def code_execution_tool(data):
    """Simple computation tool."""
    avg_price = sum(m["price"] for m in data) / len(data)
    avg_cal = sum(m["calories"] for m in data) / len(data)

    return {
        "avg_price": round(avg_price, 2),
        "avg_calories": round(avg_cal, 2)
    }


# ============================================================
#  AGENTS
# ============================================================
class Planner:
    def run(self, goal):
        prompt = f"""
        You are the Planner Agent in a Multi-Agent AI System.

        User Goal:
        - Budget: ${goal['budget']}
        - Max calories: {goal['calories']}

        Produce a short, clear 4-step plan for how the system should choose
        a healthy lunch option based on meal data, ratings, calories, and price.
        """
        return call_gemini(prompt)


class Researcher:
    def run(self):
        data = csv_tool()
        enriched = []

        for d in data:
            d["rating"] = openapi_tool("/rating", {"meal": d["meal"]})["rating"]
            enriched.append(d)

        return enriched


class Coder:
    def run(self, meals):
        return code_execution_tool(meals)


class Critic:
    def run(self, summary, goal):
        ok = (
            summary["avg_price"] <= goal["budget"]
            and summary["avg_calories"] <= goal["calories"]
        )
        return "✅ Meets goal!" if ok else "⚠️ Needs adjustment."


# ============================================================
#  ORCHESTRATOR
# ============================================================
class Orchestrator:
    def __init__(self):
        self.session = InMemorySessionService()
        self.memory = MemoryBank()
        self.state = self.memory.load() or {}

    def run(self, goal):
        print("=== Healthy Lunch Planner AI (Gemini 2.0 Enhanced) ===")

        planner = Planner()
        researcher = Researcher()
        coder = Coder()
        critic = Critic()

        print("\n📝 Planner (Gemini):")
        plan = planner.run(goal)
        print(plan)

        print("\n🔍 Researcher:")
        meals = researcher.run()
        for m in meals:
            print(m)

        print("\n💻 Coder:")
        summary = coder.run(meals)
        print(summary)

        print("\n🧩 Critic:")
        decision = critic.run(summary, goal)
        print(decision)

        # Save memory
        self.state = {
            "goal": goal,
            "plan": plan,
            "summary": summary,
            "decision": decision
        }
        self.memory.save(self.state)

        print("\n💾 Memory saved to lunch_memory.json")


# ============================================================
#  RUN SYSTEM
# ============================================================
goal = {"budget": 8.0, "calories": 600}
Orchestrator().run(goal)


=== Healthy Lunch Planner AI (Gemini 2.0 Enhanced) ===

📝 Planner (Gemini):
Offline Plan: 1) Load meal data. 2) Add ratings. 3) Compute averages. 4) Compare to user goals.

🔍 Researcher:
{'meal': 'Chicken Salad', 'calories': 450, 'price': 7.5, 'rating': 4.7}
{'meal': 'Veggie Wrap', 'calories': 380, 'price': 6.0, 'rating': 4.8}
{'meal': 'Sushi Bento', 'calories': 520, 'price': 9.0, 'rating': 4.4}
{'meal': 'Beef Bowl', 'calories': 640, 'price': 8.5, 'rating': 3.8}

💻 Coder:
{'avg_price': 7.75, 'avg_calories': 497.5}

🧩 Critic:
✅ Meets goal!

💾 Memory saved to lunch_memory.json
